In [129]:
#Aashir Hussain
#991624230
#Ai Final Project: Red Wine (Target Value = Quality, 0-10 as categories)

import pandas as pd
#Using prof provided code to download the dataset and display it
!wget -q https://archive.ics.uci.edu/static/public/186/wine+quality.zip
!unzip -q wine+quality.zip

df_wine = pd.read_csv('winequality-red.csv',sep=';')
df_wine

replace winequality-red.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace winequality-white.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace winequality.names? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [130]:
#Lets begin by dropping the quality column, since its the target value.

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler


# Separating the target value from the dataset
X = df_wine.drop('quality',axis=1)
y = df_wine['quality']

# Feature scaling to avoid unnecessary deviations since we have a lot of features.
scaler = RobustScaler() # I tried different scalers, this was the best one
X_scaled = scaler.fit_transform(X)

# Splitting the dataset into the Training set and Test set
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.25, random_state=42)

# Splitting the dataset into the Training set and Test set with stratification
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

#Checking the dispersion of the dataset NOTE: This dispersion yielded the highest accuracy.
print(f'Training set size: {X_train.shape[0]} samples')
print(f'Test set size: {X_test.shape[0]} samples')


Training set size: 1199 samples
Test set size: 400 samples


In [131]:
#MODELS I TRIED: KNN, Logistic Regression, Decision Tree, SVM, Gradient Boosting, XGBoost.


#RF was BEST performing - This specific iteration was best - Assigns weights to balance each class for better prediction.

from sklearn.utils.class_weight import compute_class_weight
import numpy as np
from sklearn.ensemble import RandomForestClassifier


class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(df_wine['quality']), y=df_wine['quality'])


class_weights_dict = {label: weight for label, weight in zip(np.unique(df_wine['quality']), class_weights)}
print(class_weights_dict)


rf_classifier = RandomForestClassifier(class_weight=class_weights_dict, random_state=42)


rf_classifier.fit(X_train, y_train)


y_pred = rf_classifier.predict(X_test)


print("Accuracy Score:", accuracy_score(y_test, y_pred))


{3: 26.65, 4: 5.028301886792453, 5: 0.39133627019089573, 6: 0.4177115987460815, 7: 1.3391959798994975, 8: 14.805555555555555}
Accuracy Score: 0.695


In [132]:
#Standard RF

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)


rf_classifier.fit(X_train, y_train)


y_pred = rf_classifier.predict(X_test)


print("Accuracy Score:", accuracy_score(y_test, y_pred))

Accuracy Score: 0.68


In [133]:
#RF with SMOTE (Over Sampling the miniority classes) + Class Weights

from sklearn.utils.class_weight import compute_class_weight
import numpy as np
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score


class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(df_wine['quality']), y=df_wine['quality'])


class_weights_dict = {label: weight for label, weight in zip(np.unique(df_wine['quality']), class_weights)}
print("Class weights:", class_weights_dict)


smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)


rf_classifier = RandomForestClassifier(class_weight=class_weights_dict, random_state=42)


rf_classifier.fit(X_train_smote, y_train_smote)


y_pred = rf_classifier.predict(X_test)


print("Accuracy Score:", accuracy_score(y_test, y_pred))


Class weights: {3: 26.65, 4: 5.028301886792453, 5: 0.39133627019089573, 6: 0.4177115987460815, 7: 1.3391959798994975, 8: 14.805555555555555}
Accuracy Score: 0.65


In [134]:
#RF using just SMOTE

from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score


smote = SMOTE(random_state=42)
X_smote, y_smote = smote.fit_resample(X_train, y_train)


rf_classifier = RandomForestClassifier(random_state=42)
rf_classifier.fit(X_smote, y_smote)


y_pred = rf_classifier.predict(X_test)


print("Accuracy:", accuracy_score(y_test, y_pred))


Accuracy: 0.635


In [135]:
#RF with selected features

selected_features = [0,1,2,3,4,5,6,7,8,9,10] #Im trying to select specific features to see if I can get higher accuracy

# Select these features from the original dataset
X_trainR= X_train[:, selected_features]
X_testR = X_test[:, selected_features]

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)


rf_classifier.fit(X_trainR, y_train)


y_pred = rf_classifier.predict(X_testR)


print("Accuracy Score:", accuracy_score(y_test, y_pred))

Accuracy Score: 0.68


In [136]:
#KNN
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score


knn_classifier = KNeighborsClassifier(n_neighbors=1) #Played with this value, the highest accuracy is at 1


knn_classifier.fit(X_train, y_train)


y_pred_knn = knn_classifier.predict(X_test)


print("KNN Accuracy Score:", accuracy_score(y_test, y_pred_knn))

KNN Accuracy Score: 0.6075


In [137]:
#Logistic Regression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


log_reg = LogisticRegression(multi_class='ovr', max_iter=1000, random_state=42)


log_reg.fit(X_train, y_train)


y_pred_log_reg = log_reg.predict(X_test)


print("Logistic Regression Accuracy Score:", accuracy_score(y_test, y_pred_log_reg))

Logistic Regression Accuracy Score: 0.5675
